## Installing dependencies

In [ ]:

!pip install 'instructlab[cuda]' \
   -C cmake.args="-DLLAMA_CUDA=on" \
   -C cmake.args="-DLLAMA_NATIVE=off"
!pip install vllm@git+https://github.com/opendatahub-io/vllm@2024.08.01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 924.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 78.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/opendatahub-io/vllm (to revision 2024.08.01) to /tmp/pip-install-ahue9rzr/vllm_2ef1cd4ac4cf44fda6c2824656c1418f
  Running command git clone --filter=blob:none --quiet https://github.com/opendatahub-io/vllm /tmp/pip-install-ahue9rzr/vllm_2ef1cd4ac4cf44fda6c2824656c1418f
  Running command git checkout -q a1ba24079bb513799ccd4a45bab26cf1c6e6ccd4
  Resolved https://github.com/opendatahub-io/vllm to commit a1ba24079bb513799ccd4a45bab26cf1c6e6ccd4
  Installing build dependencies ... canceledERROR: Operation cancelled by user
^C


## Taxonomy

Now I need to clone the taxonomy repository:

docs: https://docs.instructlab.ai/taxonomy/

In [ ]:
# Remove existing taxonomy folder if exists
!rm -rf /taxonomy

# Cloning the taxonomy repository
!git clone https://github.com/instructlab/taxonomy.git

## Initialize the instructlab

Now I need to initialize the **instructlab**. Remember to get the taxonomy cloned repository absolut path - in my case, it is `/content/taxonomy`.

Note: If yout are in Jupyter notebook you will not be able to interact with the output. If so, run the next command in the terminal.

In [ ]:
!ilab config init --non-interactive

## Download the models

In this case we will be using a 2B model.

In [ ]:
# Download with safetensors
!ilab model download --repository ibm-granite/granite-3.3-2b-instruct-GGUF --filename granite-3.3-2b-instruct-Q4_K_M.gguf

## Synthetic Data

Lets clone the minimodel repository.

I've created inside the minimodel repo the `knowledge/taxonomy` folder with `minima_qna.yaml` following the knowledge template provided by the `taxonomy` repository.

The `minima_qna.yaml` file contains the questions and answers extracted from the minima whitepaper and brand
 guidelines.

In [ ]:
# Clone the minimodel repository
!git clone https://github.com/akio-code/minimodel-fine-tuning /minimodel

### Copying the minima taxonomy

Now we need to copy the `minima_qna.yaml` and `attribution.txt` to the **taxonomy** repository. It will be use to generate the data with the instructlab.

In [ ]:
# Create the destination folder
!mkdir -p ./taxonomy/knowledge/business

# Copy from minimodel
!cp ./minimodel/knowledge/taxonomy/minima_qna.yaml ./taxonomy/knowledge/business/qna.yaml
!cp ./minimodel/knowledge/taxonomy/attribution.txt ./taxonomy/knowledge/business/

Now I check if the taxonomy is valid:

In [ ]:
!ilab taxonomy diff

## Generating the data

docs: https://docs.instructlab.ai/adding-data-to-model/creating_new_knowledge_or_skills/

In [ ]:
# Generating the synthetic data
!ilab data generate \
--taxonomy-path ./taxonomy \
--output-dir ./datasets \
--model /root/.cache/instructlab/models/granite-3.3-2b-instruct-Q4_K_M.gguf \
--pipeline simple \
--sdg-scale-factor 1